In [ ]:
import pandas as pd
import numpy as np
# raw = pd.read_csv('/content/drive/MyDrive/data_science_machine_learning/project_box_data_science/1_new_models_since_raw_data/raw_data_wiki_bets_no_procesed_23-9-30.csv')

In [ ]:
def cleaner_raw_data(df):
  '''
  Function to clean the raw data of boxers records.
    Parameters:
      df: Raw df to clean.
    Returns:
      df: Cleaned with procesed odds to make them decimal, removed incoherent odds values,
          duplicated values from result, shuffled and removed incoherent values from draws.
  '''
  raw = df.copy()
  # 1er elimino columnas
  raw = raw.drop(columns=['id_b1','id_b2','d_bet'])
  # 2do separo para manipular
  raw = raw[['result', 'endtype', 'endround', 'boxer1', 'boxer2',
      'y_f', 'm_f','d_f', 'c_f',
      'birth_b1', 'height_b1', 'reach_b1','stance_b1', 'boxstyle_b1', 'region_b1',
      'birth_b2','height_b2', 'reach_b2', 'stance_b2', 'boxstyle_b2', 'region_b2',
      'b1_w', 'b1_wk', 'b1_d', 'b1_l', 'b1_lk','b1_bet',
      'text_boxer1', 'summary_boxer1','len_text_boxer1', 'len_summary_boxer1',
      'b2_w', 'b2_wk', 'b2_d','b2_l', 'b2_lk', 'b2_bet',
      'text_boxer2','summary_boxer2', 'len_text_boxer2', 'len_summary_boxer2']]

  # creacion de funcion odds cleaner
  def odds_cleaner(column):
    # procesar registros con :
    lista = []
    for i in column:
      if ':' in i:
        e = i.split(':')
        try:
          num = float(e[0]) / float(e[1])
          lista.append(num)
        except:
          lista.append(e[1]) # e[0] no tenia valor numerico
      else:
        lista.append(i)
    # procesar registros con /, y american odds positivos
    lista2 = []
    for i in lista:
      i = str(i).replace('--','-').replace('-','/')
      if '/' in str(i):
        if i[0] == '/':
          try:
            num = i.replace('/','').strip()
            num = (100 / float(num)) + 1
            lista2.append(num)
          except:
            lista2.append(i)
        else:
          e = i.split('/')
          try:
            if (float(e[0]) < 3) & (float(e[1]) == 1):
              num = e[0]
            elif (float(e[1]) > float(e[0])):
              num = (float(e[0]) + float(e[1])) / float(e[1])
            elif float(e[1]) > 1:
              num = float(e[0]) / float(e[1])
            else:
              num = (float(e[0]) / 100) + 1
            lista2.append(num)
          except:
            lista2.append(e[0]) # solo un valor tenia un valor decimal coherente
      else:
        lista2.append(i)
    # Procesar registros con American odds positivos
    lista3 = []
    for i in lista2:
      try:
        i = str(i).strip()
        if (i[0] == '+') or (float(i) > 99):
          i = i.replace('+','')
          if float(i) < 100:
            lista3.append(i)
          else:
            num = (float(i) / 100) + 1
            lista3.append(num)
        else:
          lista3.append(i)
      except:
        lista3.append(i)
    return lista3

  # aplico funcion cleaner
  raw['b1_bet'] = pd.to_numeric(odds_cleaner(raw.b1_bet), errors='coerce')
  raw['b2_bet'] = pd.to_numeric(odds_cleaner(raw.b2_bet), errors='coerce')

  raw = raw[(raw.b1_bet.notna()) & (raw.b2_bet.notna())] # quito valores nulos
  raw = raw[(raw.b1_bet < 3) & (raw.b2_bet < 3)] # quito valores altos incoherentes
  raw = raw[(raw.b1_bet > 1) & (raw.b2_bet > 1)] # quito odds negativos

  # Calculate the difference in counts
  win_count = (raw['result'] == 'win').sum()
  loss_count = (raw['result'] == 'loss').sum()
  diff = abs(win_count - loss_count)
  # Duplicate rows with "loss" to balance the counts
  if win_count > loss_count:
      df_to_append = raw[raw['result'] == 'loss'].sample(n=diff, replace=True)
  else:
      df_to_append = raw[raw['result'] == 'win'].sample(n=diff, replace=True)
  # Concatenate the original DataFrame with the new one
  raw = pd.concat([raw, df_to_append], ignore_index=True)

  raw = raw[raw.result.str.contains("win|loss")] # remove result nc and draws
  raw = raw.sample(frac=1).reset_index(drop=True) # aleatorizar las filas del DataFrame
  raw = raw[(raw.b1_d < 10) & (raw.b2_d < 10)] # quito valores de empates incoherentes

  # fill nan values
  raw['text_boxer1'] = raw.text_boxer1.fillna('')
  raw['summary_boxer1'] = raw.summary_boxer1.fillna('')
  raw['text_boxer2'] = raw.text_boxer2.fillna('')
  raw['summary_boxer2'] = raw.summary_boxer2.fillna('')
  raw['len_text_boxer2'] = raw.len_text_boxer2.fillna(0)
  raw['len_summary_boxer2'] = raw.len_summary_boxer2.fillna(0)

  return raw

In [ ]:
# raw.query('b2_d > 10')[['result','boxer1','boxer2','y_f','b1_d','b2_d']]